# Good hint Migration 
This notebook demonstrates how an LLM performs when presented with a Spring Security migration issue with a good example

Installing pre requisites and configuring Kai with `Llama-3.1-8B-Instruct`

In [ ]:
%pip uninstall kai -y
%pip install --no-cache-dir git+https://github.com/konveyor/kai.git@main
%pip install python-dotenv

In [ ]:
from IPython.display import display, Markdown
from kai.llm_interfacing.model_provider import ModelProvider
from kai.kai_config import KaiConfigModels, SupportedModelProviders
from dotenv import load_dotenv
import os
load_dotenv(override=True) 

# Initialize the model provider using Llama-3.1-8B-Instruct via OpenAI
model = ModelProvider.from_config(KaiConfigModels(
    provider=SupportedModelProviders.CHAT_OPENAI,
    args={"model": "meta-llama/Llama-3.1-8B-Instruct",
        "base_url": "https://llama-3-1-8b-instruct-maas-apicast-production.apps.prod.rhoai.rh-aiservices-bu.com:443/v1",
        "api_key": os.getenv("OPENAI_API_KEY"),},
))

# Async rendering function for displaying the response
async def rendered_llm_call(prompt: str):
    response = await model.ainvoke_llm(prompt)
    display(Markdown(response.content))
    return response



/Users/sraghuna/local_dev/kai-notebook-solution-server/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Below is the snippet we are trying to migrate from springboot2 to springboot3. This code uses the discouraged `configure(AuthenticationManagerBuilder)` method inside a `WebSecurityConfigurerAdapter`, both of which are deprecated in Spring Security 5.7+ and removed in Spring Boot 3.

Our goal is to assess how well the LLM refactors this configuration without a hint, and whether it can replace deprecated patterns with the right approach.

In [3]:
before_code = """\
import org.springframework.context.annotation.Bean;
import org.springframework.context.annotation.Configuration;
import org.springframework.security.config.annotation.authentication.builders.AuthenticationManagerBuilder;
import org.springframework.security.config.annotation.web.configuration.EnableWebSecurity;
import org.springframework.security.config.annotation.web.configuration.WebSecurityConfigurerAdapter;
import org.springframework.security.crypto.bcrypt.BCryptPasswordEncoder;
import org.springframework.security.crypto.password.PasswordEncoder;

@Configuration
@EnableWebSecurity
public class SecurityConfig extends WebSecurityConfigurerAdapter {
    @Override
    protected void configure(AuthenticationManagerBuilder auth) throws Exception {
        auth.inMemoryAuthentication()
            .withUser("user")
            .password(passwordEncoder().encode("password"))
            .roles("USER");
    }

    @Bean
    public PasswordEncoder passwordEncoder() {
        return new BCryptPasswordEncoder();
    }
}
"""

This rule flags the use of `configure(AuthenticationManagerBuilder)`. In Spring Boot 3, developers should instead use a `UserDetailsService` and `PasswordEncoder` bean for authentication configuration.

In [4]:
hint_to_migrate="""\
    
  description: configure(AuthenticationManagerBuilder) override is discouraged since Spring Security 5.7.
  message: |
    Overriding `configure(AuthenticationManagerBuilder)` is **discouraged** in Spring Security 5.7.
    Instead, define a `UserDetailsService` and `PasswordEncoder` bean.
    Use a SecurityFilterChain bean to configure HTTP security instead of extending WebSecurityConfigurerAdapter

    """

Similar example showing before and after migration

In [5]:
example_before_code="""\

@Configuration
public class SecurityConfiguration extends WebSecurityConfigurerAdapter {
    @Override
    protected void configure(AuthenticationManagerBuilder auth) throws Exception {
        UserDetails user = User.withDefaultPasswordEncoder()
            .username("user")
            .password("password")
            .roles("USER")
            .build();
        auth.inMemoryAuthentication()
            .withUser(user);
    }
}

"""

example_after_code="""\
@Configuration
public class SecurityConfiguration {
    @Bean
    public InMemoryUserDetailsManager userDetailsService() {
        UserDetails user = User.withDefaultPasswordEncoder()
            .username("user")
            .password("password")
            .roles("USER")
            .build();
        return new InMemoryUserDetailsManager(user);
    }
}
"""

summary_of_changes="""\
* Removed the WebSecurityConfigurerAdapter class.
* Created a UserDetailsService bean instead of overriding configure(AuthenticationManagerBuilder).
* Defined a SecurityFilterChain bean to handle HTTP security configurations.
* Kept the PasswordEncoder bean for password encoding.

"""

In [6]:
prompt = f"""

The following java code needs to be migrated from springboot2 to springboot3.

```java
{before_code}
```
I will provide you with an example before/after code migration and a summary of changes
```java
{example_before_code}
{example_after_code}
```

{summary_of_changes}

Use the following hint's description and message to migrate. 

{hint_to_migrate}

Also, remove the deprecated and unused APIs and suggest the correct alternatives based on the migration needs. 

Can you provide a refactored version of {before_code} that is compatible with springboot3?
Also, create a step by step process for the above migration? Summarize and create a hint that supplements the data in the prompt

"""

In [7]:

response = await rendered_llm_call(prompt)

Here's the refactored version of the code that is compatible with Spring Boot 3:

```java
import org.springframework.context.annotation.Bean;
import org.springframework.context.annotation.Configuration;
import org.springframework.security.config.annotation.web.builders.HttpSecurity;
import org.springframework.security.config.annotation.web.configuration.EnableWebSecurity;
import org.springframework.security.config.annotation.web.configuration.WebSecurityConfigurerAdapter;
import org.springframework.security.crypto.bcrypt.BCryptPasswordEncoder;
import org.springframework.security.crypto.password.PasswordEncoder;

@Configuration
@EnableWebSecurity
public class SecurityConfig {
    @Bean
    public PasswordEncoder passwordEncoder() {
        return new BCryptPasswordEncoder();
    }

    @Bean
    public SecurityFilterChain securityFilterChain(HttpSecurity http) throws Exception {
        http.authorizeRequests()
            .antMatchers("/").permitAll()
            .anyRequest().authenticated()
            .and()
            .httpBasic();
        return http.build();
    }
}
```

However, the recommended approach in Spring Security 5.7 and later is to use the `SecurityFilterChain` bean directly without extending `WebSecurityConfigurerAdapter`. Here's the updated code:

```java
import org.springframework.context.annotation.Bean;
import org.springframework.context.annotation.Configuration;
import org.springframework.security.config.annotation.web.builders.HttpSecurity;
import org.springframework.security.config.annotation.web.configuration.EnableWebSecurity;
import org.springframework.security.crypto.bcrypt.BCryptPasswordEncoder;
import org.springframework.security.crypto.password.PasswordEncoder;

@Configuration
@EnableWebSecurity
public class SecurityConfig {
    @Bean
    public PasswordEncoder passwordEncoder() {
        return new BCryptPasswordEncoder();
    }

    @Bean
    public SecurityFilterChain securityFilterChain(HttpSecurity http) throws Exception {
        http.authorizeRequests()
            .antMatchers("/").permitAll()
            .anyRequest().authenticated()
            .and()
            .httpBasic();
        return http.build();
    }
}
```

Here's a step-by-step process for the migration:

**Step 1: Remove the `WebSecurityConfigurerAdapter` class**

* Remove the `extends WebSecurityConfigurerAdapter` part from the `SecurityConfig` class.

**Step 2: Define a `UserDetailsService` bean**

* Create a new class that implements `UserDetailsService`.
* Use the `User.withDefaultPasswordEncoder()` method to create a user.
* Return the user from the `loadUserByUsername()` method.

**Step 3: Define a `PasswordEncoder` bean**

* Create a new method that returns a `PasswordEncoder` bean.
* Use the `BCryptPasswordEncoder` class to create a password encoder.

**Step 4: Define a `SecurityFilterChain` bean**

* Create a new method that returns a `SecurityFilterChain` bean.
* Use the `HttpSecurity` object to configure the security settings.
* Use the `authorizeRequests()` method to define the authorization rules.
* Use the `httpBasic()` method to enable HTTP basic authentication.

**Step 5: Remove the deprecated `configure(AuthenticationManagerBuilder)` method**

* Remove the `configure(AuthenticationManagerBuilder)` method from the `SecurityConfig` class.

**Step 6: Update the `SecurityConfig` class**

* Update the `SecurityConfig` class to use the new `SecurityFilterChain` bean.
* Remove any unnecessary code.

Here's a hint that supplements the data in the prompt:

**Hint:** When migrating from Spring Security 5.6 to 5.7, you should use the `SecurityFilterChain` bean directly instead of extending `WebSecurityConfigurerAdapter`. You should also define a `UserDetailsService` bean and a `PasswordEncoder` bean separately.

**Description:** When migrating from Spring Security 5.6 to 5.7, you should use the `SecurityFilterChain` bean directly instead of extending `WebSecurityConfigurerAdapter`. You should also define a `UserDetailsService` bean and a `PasswordEncoder` bean separately.

**Message:** 
Overriding `configure(AuthenticationManagerBuilder)` is **discouraged** in Spring Security 5.7.
Instead, define a `UserDetailsService` and `PasswordEncoder` bean.
Use a SecurityFilterChain bean to configure HTTP security instead of extending WebSecurityConfigurerAdapter.

### Evaluation

| Criteria                                                                 | Result         |
|--------------------------------------------------------------------------|----------------|
| Removed deprecated `configure(AuthenticationManagerBuilder)`            |  Yes           |
| Replaced with `UserDetailsService` and `PasswordEncoder` bean           |  Yes |
| Removed `WebSecurityConfigurerAdapter`                                   |  Yes           |
| The output compiles                                                      | <todo>          |
| Preserves behavior (in-memory auth, user with role)                      | Yes        |